Managing kubernetes objects using common resource operations with the python client
-----------------------------------------------------------------------------------------------

Some of these operations include;

- **`create_xxxx`** : create a resource object. Ex **`create_namespaced_pod`** and **`create_namespaced_deployment`**, for creation of pods and deployments respectively. This performs operations similar to **`kubectl create`**.


- **`read_xxxx`** : read the specified resource object. Ex **`read_namespaced_pod`** and **`read_namespaced_deployment`**, to read pods and deployments respectively. This performs operations similar to **`kubectl describe`**.


- **`list_xxxx`** : retrieve all resource objects of a specific type. Ex **`list_namespaced_pod`** and **`list_namespaced_deployment`**, to list pods and deployments respectively. This performs operations similar to **`kubectl get`**.


- **`patch_xxxx`** : apply a change to a specific field. Ex **`patch_namespaced_pod`** and **`patch_namespaced_deployment`**, to update pods and deployments respectively. This performs operations similar to **`kubectl patch`**, **`kubectl label`**, **`kubectl annotate`** etc.


- **`replace_xxxx`** : replacing a resource object will update the resource by replacing the existing spec with the provided one. Ex **`replace_namespaced_pod`** and **`replace_namespaced_deployment`**, to update pods and deployments respectively, by creating new replacements of the entire object. This performs operations similar to **`kubectl rolling-update`**, **`kubectl apply`** and **`kubectl replace`**.


- **`delete_xxxx`** : delete a resource. This performs operations similar to **`kubectl delete`**.


For Futher information see the Documentation for API Endpoints section in https://github.com/kubernetes-client/python/blob/master/kubernetes/README.md

In [1]:
from kubernetes import client, config

In [2]:
config.load_kube_config()

In [5]:
api_instance = client.AppsV1Api()
# the following differs from the broken example 
name = "my-busybox"
spec = client.V1DeploymentSpec(
    selector=client.V1LabelSelector(match_labels={"app":"busybox"}),
    template=client.V1PodTemplateSpec(),
)
container = client.V1Container(
    image="busybox:1.26.1",
    args=["sleep", "3600"],
    name=name,
)
spec.template.metadata = client.V1ObjectMeta(
    name="busybox",
    labels={"app":"busybox"},
)
spec.template.spec = client.V1PodSpec(containers = [container])
dep = client.V1Deployment(
    metadata=client.V1ObjectMeta(name=name),
    spec=spec,
)

In [6]:
api_instance.create_namespaced_deployment(namespace="default",body=dep)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 12, 16, 12, 45, 12, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': None,
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:progressDeadlineSeconds': {},
                                                           'f:replicas': {},
                                                           'f:revisionHistoryLimit': {},
                                                           'f:selector': {'f:matchLabels': {'.': {},
                                                                           

In [7]:
deps = api_instance.list_namespaced_deployment(namespace="default")
for item in deps.items:
    print("%s  %s" % (item.metadata.namespace, item.metadata.name))

default  grafana
default  my-busybox
default  prometheus-alertmanager
default  prometheus-kube-state-metrics
default  prometheus-pushgateway
default  prometheus-server


In [8]:
api_instance.read_namespaced_deployment(namespace="default",name=name)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': {'deployment.kubernetes.io/revision': '1'},
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 12, 16, 12, 45, 12, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': None,
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:progressDeadlineSeconds': {},
                                                           'f:replicas': {},
                                                           'f:revisionHistoryLimit': {},
                                                           'f:selector': {'f:matchLabels': {'.': {},
                                     

In [9]:
dep.metadata.labels = {"key": "value"}
api_instance.patch_namespaced_deployment(name=name, namespace="default", body=dep)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': {'deployment.kubernetes.io/revision': '1'},
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 12, 16, 12, 45, 12, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': {'key': 'value'},
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:metadata': {'f:annotations': {'.': {},
                                                                                 'f:deployment.kubernetes.io/revision': {}}},
                                                'f:status': {'f:availableReplicas': {},
                                                             'f:conditions': {'.

In [10]:
dep.spec.template.spec.containers[0].image = "busybox:1.26.2"
api_instance.replace_namespaced_deployment(name=name, namespace="default", body=dep)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 12, 16, 12, 45, 12, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 2,
              'labels': {'key': 'value'},
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:metadata': {'f:labels': {'.': {},
                                                                            'f:key': {}}},
                                                'f:spec': {'f:progressDeadlineSeconds': {},
                                                           'f:replicas': {},
                                                           'f:revisionHistory

In [11]:
api_instance.delete_namespaced_deployment(name=name, namespace="default", body=client.V1DeleteOptions(propagation_policy="Foreground", grace_period_seconds=5))

{'api_version': 'apps/v1',
 'code': None,
 'details': None,
 'kind': 'Deployment',
 'message': None,
 'metadata': {'_continue': None,
              'remaining_item_count': None,
              'resource_version': '100740',
              'self_link': '/apis/apps/v1/namespaces/default/deployments/my-busybox'},
 'reason': None,
 'status': "{'observedGeneration': 2, 'replicas': 2, 'updatedReplicas': 1, "
           "'readyReplicas': 1, 'availableReplicas': 1, 'unavailableReplicas': "
           "1, 'conditions': [{'type': 'Available', 'status': 'True', "
           "'lastUpdateTime': '2020-12-16T12:45:14Z', 'lastTransitionTime': "
           "'2020-12-16T12:45:14Z', 'reason': 'MinimumReplicasAvailable', "
           "'message': 'Deployment has minimum availability.'}, {'type': "
           "'Progressing', 'status': 'True', 'lastUpdateTime': "
           "'2020-12-16T12:45:33Z', 'lastTransitionTime': "
           "'2020-12-16T12:45:12Z', 'reason': 'ReplicaSetUpdated', 'message': "
          